In [1]:
import os
from dotenv import load_dotenv

load_dotenv("./credentials.env")

True

In [2]:
# Chat model
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name=os.getenv("OPENAI_MODEL_NAME"),
    openai_api_base=os.getenv("OPENAI_API_BASE"),
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0,
    model_kwargs={
        "deployment_id": os.getenv("OPENAI_DEPLOYMENT_ID"),
        "api_type": os.getenv("OPENAI_API_TYPE"),
        "api_version": os.getenv("OPENAI_API_VERSION"),
    },
)

In [3]:
from langchain.document_loaders import YoutubeLoader

def load_documents(url):
    return YoutubeLoader.from_youtube_url(
        url,
        add_video_info=True,
        language=['en', 'ja']
    ).load()

In [4]:
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback
from langchain.chains.summarize import load_summarize_chain

PROMPT_TEMPLATE = """Write a concise Japanese summary of the following transcript of Youtube Video.
============
    
{text}

============

日本語で 500 文字以内で要約した結果は以下の通りです。
"""

def summarize(docs, prompt_template=PROMPT_TEMPLATE):
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
    
    with get_openai_callback() as cb:
        chain = load_summarize_chain( 
            llm,
            chain_type="stuff",
            verbose=False,
            prompt=PROMPT
        )
        response = chain({"input_documents": docs}, return_only_outputs=True)
    return response['output_text'], cb.total_cost

In [5]:
url = "https://www.youtube.com/watch?v=FZhbJZEgKQ4"

In [6]:
docs = load_documents(url=url)
output, cost = summarize(docs)

In [7]:
output, cost

('この動画の要約は、AIと混合現実、およびAIと量子コンピューティングの分野での革新に焦点を当てています。Microsoft Copilotは、人々が仕事や学習をより効果的に行うためのコパイロットとしての役割を果たします。Copilotは、検索、開発、顧客サービスなど、さまざまな役割とビジネス機能にわたって拡張されています。また、Copilot Studioを使用することで、組織固有のニーズに合わせてカスタマイズすることも可能です。さらに、AIと量子コンピューティングの組み合わせにより、科学的な発見や複雑なシミュレーションの高速化が可能になります。最終的には、AIを通じて、個々の人や組織がより能力を発揮し、より良い未来を築くことができるという使命を持っています。',
 0.031907)